In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [20]:
import os

In [14]:
class nn_model(nn.Module):
    def __init__(self, input_shape, output_shape):
        super(nn_model, self).__init__()
        self.fc1 = nn.Linear(input_shape, 32)
        self.fc2 = nn.Linear(32, output_shape)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        return self.fc2(x)
    

In [16]:
nn_model(28*28, 10)(torch.rand(32, 28*28)).shape

torch.Size([32, 10])

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [18]:
in_size = 28*28
out_size = 10
learning_rate = 0.001
batch_size= 32
epochs = 5

['.ipynb_checkpoints',
 'data',
 'first_tensor.t',
 'pytorch_leanring_2.ipynb',
 'pytorch_learning.ipynb',
 'pytorch_learning3.ipynb',
 'runs']

In [23]:
train_set = datasets.MNIST(root='data/', train=True, transform = transforms.ToTensor(), download=True)

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting data/MNIST\raw\train-images-idx3-ubyte.gz to data/MNIST\raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting data/MNIST\raw\train-labels-idx1-ubyte.gz to data/MNIST\raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting data/MNIST\raw\t10k-images-idx3-ubyte.gz to data/MNIST\raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting data/MNIST\raw\t10k-labels-idx1-ubyte.gz to data/MNIST\raw



C:\Users\An_asus\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [28]:
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)

In [30]:
test_set = datasets.MNIST(root='data/', train=False, transform = transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)

In [31]:
model = nn_model(input_shape=in_size, output_shape=out_size).to(device)

In [35]:
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [39]:
for epoch in range(epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        
        #moving data to cuda
        data = data.to(device=device)
        targets = targets.to(device=device)
        
        #reshaping data
        data = data.view(data.shape[0], -1)
        
        #forward
        scores = model(data)
        
        #calculate loss
        losses = loss(scores, targets)
        
        #backward
        optimizer.zero_grad()
        losses.backward()
        
        #gradient
        optimizer.step()
        
        

In [45]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("checking accuracy of the training data")
    else:
        print("checking accuracy of the test data")
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            
            x = x.view(x.shape[0], -1)
            
            scores = model(x)
            
            _, preds = scores.max(1)
            
            num_correct += (preds == y).sum()
            
            num_samples += preds.size(0)
        print(f'got {num_correct} / {num_samples} with accuracy {float(num_correct) / float(num_samples)*100:.2f}')
    
    model.train()
            

In [46]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

checking accuracy of the training data
got 58103 / 60000 with accuracy 96.84
checking accuracy of the test data
got 9608 / 10000 with accuracy 96.08


In [62]:
train_fashion_set = datasets.FashionMNIST(root='data/', train=True, transform=transforms.ToTensor(), download=True)
train_fashion_loader = DataLoader(dataset=train_fashion_set, batch_size=batch_size, shuffle=True)

test_fashion_set = datasets.FashionMNIST(root='data/', train=False, transform=transforms.ToTensor(), download=True)
test_fashion_loader = DataLoader(dataset=test_fashion_set , batch_size=batch_size, shuffle=True)

In [73]:
class CNN(nn.Module):
    def __init__(self, in_shape, out_shape):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_shape, out_channels=16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.fc1 = nn.Linear(32 * 7 * 7, 64)
        self.fc2 = nn.Linear(64, out_shape)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(self.pool(x))))
        
        x = x.reshape(x.shape[0], -1)
        
        return self.fc2(F.relu(self.fc1(x)))

In [74]:
cnn_model = CNN(in_shape=1, out_shape=10).to(device)

In [75]:
cnn_model(torch.rand((64, 1, 28, 28)).to(device)).shape

torch.Size([64, 10])

In [76]:
criterion = nn.CrossEntropyLoss()
optimizer_cnn = optim.Adam(cnn_model.parameters(), lr=learning_rate)

In [77]:
for epoch in range(epochs):
    
    for data_idx, (data, targets) in enumerate(train_fashion_loader):
        
        data = data.to(device)
        
        targets = targets.to(device)
        
        scores = cnn_model(data)
        
        loss = criterion(scores, targets)
        
        optimizer_cnn.zero_grad()
        
        loss.backward()
        
        optimizer_cnn.step()       

In [88]:
def check_accuracy_cnn(loader, model):
    if loader.dataset.train:
        print("checking accuracy of the training data")
    else:
        print("checking accuracy of the test data")
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            
            
            scores = model(x)
            
            _, preds = scores.max(1)
            
            num_correct += (preds == y).sum()
            
            num_samples += preds.size(0)
        print(f'got {num_correct} / {num_samples} with accuracy {float(num_correct) / float(num_samples)*100:.2f}')
    
    model.train()

In [90]:
check_accuracy_cnn(model=cnn_model, loader=train_fashion_loader)
check_accuracy_cnn(model=cnn_model, loader=test_fashion_loader)

checking accuracy of the training data
got 55733 / 60000 with accuracy 92.89
checking accuracy of the test data
got 9094 / 10000 with accuracy 90.94
